# 🔍 Data Discovery: fact_educacion & fact_calidad_ambiental

**Fecha:** 15 de diciembre de 2025  
**Autor:** Barcelona Housing Analytics Team  
**Issue:** [#216](https://github.com/prototyp33/barcelona-housing-demographics-analyzer/issues/216)  
**Status:** 🚧 Research en progreso

---

## 📌 Contexto del Proyecto

Este notebook documenta la **investigación de viabilidad** para dos nuevas tablas fact prioritarias en Fase 2.1:

1. **`fact_educacion`**: Nivel educativo por barrio (proxy de gentrificación - Paper #2)
2. **`fact_calidad_ambiental`**: Contaminación aire + ruido (impacto precios - Paper #5)

### Alineación con Roadmap

| Issue | Título | Dependencia |
|-------|--------|-------------|
| #214 | K-Means Segmentation v0 | ✅ DONE |
| **#216** | **Research Data Sources** | ⏳ **EN PROGRESO** |
| #217 | Prototype Schema Experimental | 🔒 Bloqueada por #216 |
| #218 | Bayesian Causal Analysis | 🔒 Bloqueada por #217 |

### Objetivo

**Responder pregunta GO/NO-GO:** ¿Existen fuentes de datos públicas viables (granularidad barrio, cobertura ≥8 años, descargables) para implementar estas tablas fact?

---

## 🎯 Criterios de Éxito

### Must-Have (GO):
- ✅ **Granularidad:** Nivel barrio (73 barrios) **O** distrito (10) con método desagregación
- ✅ **Cobertura temporal:** ≥8 años (2015-2023 mínimo)
- ✅ **Campos clave disponibles:**
  - Educación: `universitarios_pct` (% población con título universitario)
  - Ambiental: `no2_media`, `pm25_media`, `ruido_dia_db`
- ✅ **Accesibilidad:** API REST **O** CSV descargable (no solo dashboards web)

### Deal-Breakers (NO-GO):
- ❌ Solo datos municipales (sin desagregación posible)
- ❌ Cobertura < 5 años
- ❌ Datos no descargables (solo visualizaciones embebidas)
- ❌ Actualizaciones descontinuadas (última actualización >3 años)

---



## 1️⃣ Pregunta de Decisión: GO / NO-GO

### 1.1 Formulación del Problema

**Pregunta Central:**  
> ¿Podemos obtener datos de **educación** y **calidad ambiental** a nivel barrio en Barcelona con suficiente calidad y cobertura para alimentar modelos causales (Bayesian Networks) y análisis de equidad habitacional?

### 1.2 Escenarios de Decisión

#### **Escenario A: GO para ambas tablas** ✅
**Consecuencias:**
- **Issue #217:** Proceder con diseño de schema experimental (`database_experimental.db`)
- **Issue #218:** Bayesian Network incluye `universitarios_pct` como variable causal clave
- **Análisis de equidad:** Cruzar calidad aire + ruido con precios → identificar "zonas de sacrificio"
- **Timeline:** Fase 2.1 continúa según plan (4-8 semanas)

**Criterio GO:**
- `fact_educacion`: ✅ Fuente con granularidad barrio/distrito + ≥8 años
- `fact_calidad_ambiental`: ✅ ≥10 sensores aire + estrategia agregación validada

---

#### **Escenario B: GO parcial (solo 1 tabla)** ⚠️
**Consecuencias:**
- **Si GO educación, NO-GO ambiental:**
  - Proceder con `fact_educacion` en #217
  - Bayesian Network (#218) incluye educación pero omite variables ambientales
  - Análisis de equidad limitado (sin calidad aire/ruido)
  
- **Si NO-GO educación, GO ambiental:**
  - Proceder con `fact_calidad_ambiental` en #217
  - Bayesian Network (#218) usa solo renta como proxy socioeconómico (menos robusto)
  - Análisis de equidad posible pero sin componente educativo

**Criterio GO parcial:**
- Una tabla cumple criterios, otra no
- Priorizar según impacto: **educación > ambiental** (Paper #2 lo requiere)

---

#### **Escenario C: NO-GO ambas tablas** ❌
**Consecuencias:**
- **Issue #217:** Cancelada o pospuesta a Fase 3
- **Issue #218:** Bayesian Network simplificado:
  - Solo variables actuales: `precio`, `renta`, `porc_inmigracion`, `distancia_centro`
  - Estructura DAG más débil (menos variables confounding)
  - Paper #2 tendría limitaciones significativas
- **Pivot estratégico:** Enfocar Fase 2.2 en análisis espacial (Paper #4) en vez de causal

**Criterio NO-GO:**
- Ninguna fuente viable
- Alternativa: Buscar datos privados (Idealista, INE microdata) → requiere acuerdos

---

### 1.3 Impacto en Papers Académicos

| Paper | Dependencia Educación | Dependencia Ambiental | Impacto NO-GO |
|-------|----------------------|----------------------|---------------|
| **Paper #1** (K-Means) | Opcional (enriquecimiento) | Opcional | Bajo - ya completo con #214 |
| **Paper #2** (Bayesian) | **CRÍTICA** ✅ | Media | Alto - DAG incompleto sin educación |
| **Paper #3** (TDA Gentrification) | Alta | Baja | Alto - educación es proxy gentrificación |
| **Paper #4** (Spatial Econ) | Baja | Media | Medio - *spillovers* ambientales interesantes |
| **Paper #5** (Air Quality Hedonic) | Baja | **CRÍTICA** ✅ | Alto - requiere NO₂/PM2.5 |

**Conclusión:** Educación es **más crítica** que ambiental para roadmap Papers #2-#3.

---

### 1.4 Métricas de Calidad de Datos

Evaluaremos cada fuente candidata con este *scoring*:

| Criterio | Peso | Scoring |
|----------|------|---------|
| **Granularidad** | 30% | Barrio=5, Distrito=3, Municipal=0 |
| **Cobertura Temporal** | 25% | ≥10 años=5, 8-9 años=4, 5-7 años=2, <5 años=0 |
| **Completitud Espacial** | 20% | ≥70/73 barrios=5, 60-69=4, 50-59=2, <50=0 |
| **Accesibilidad** | 15% | API pública=5, CSV directo=4, Scraping=2, Manual=0 |
| **Actualización** | 10% | Activa (≤1 año)=5, Reciente (1-2 años)=3, Antigua (>2 años)=0 |

**Score mínimo para GO:** ≥3.5/5.0 (70%)

---

### 1.5 Timeline de Decisión

```text
┌─────────────────────────────────────────────────────────────┐
│ Hoy (Dec 15)          │ Dec 20          │ Jan 3           │
├───────────────────────┼─────────────────┼──────────────────┤
│ ▶ Identificar fuentes │ ▶ Descargar     │ ▶ Decisión      │
│   candidatas          │   muestras      │   GO/NO-GO      │
│ ▶ Validar granularidad│ ▶ Validar       │ ▶ Documentar en │
│ ▶ Probar APIs/CSV     │   estructura    │   issue #216    │
└─────────────────────────────────────────────────────────────┘
                                              ▼
                                          GO → #217 (schema)
                                          NO-GO → Pivot a Paper #4
```

**Deadline decisión:** **3 de enero de 2026** (fin Sprint Fase 2.1)



---

## 2️⃣ Revisión de Documentación Interna

### 2.1 Estado Actual del Proyecto (Fuente: `PROJECT_STATUS.md`)

#### Tablas Existentes en `database.db`

| Tabla | Registros | Cobertura Temporal | Status |
|-------|-----------|-------------------|--------|
| `dim_barrios` | 73 | Estático | ✅ Completo |
| `fact_precios` | 6,358 | 2012-2025 (14 años) | ✅ Bueno |
| `fact_demografia` | 657 | 2015-2023 (9 años) | ✅ Bueno |
| `fact_demografia_ampliada` | 2,256 | 2025 (edad/sexo/nacionalidad) | ⚠️ Solo 1 año |
| `fact_renta` | 73 | 2022 (1 año) | ❌ Crítico |
| `fact_oferta_idealista` | 0 | Vacía | ❌ Pendiente |

**Análisis:**
- ✅ **Tenemos:** Población, precios (venta/alquiler), renta 2022
- ❌ **NO tenemos:** Educación, calidad ambiental, renta histórica (≤2015)

---

### 2.2 Requisitos Documentados (Fuente: `QUE_DATOS_NECESITAMOS.md`)

#### **fact_educacion** (Sección 4: "Situación Económica de los Barrios")

**Especificaciones originales:**
```
Categorías necesarias:
- Sin estudios / Primaria incompleta
- Primaria completa
- Secundaria (ESO)
- Bachillerato / FP
- Universitaria (grado)
- Universitaria (postgrado)

Granularidad temporal: Año (cada 2-3 años, cambia poco)
Rango: 2015-2024
Formato: Porcentaje de población por nivel educativo
Fuente sugerida: Censo, Encuesta de Población Activa (EPA)
```

**Campos críticos para Paper #2 (Bayesian Causal):**
- `universitarios_pct`: % población con título universitario (proxy gentrificación)
- `sin_estudios_pct`: % sin estudios (proxy vulnerabilidad)
- `secundaria_pct`: % estudios secundarios

---

#### **fact_calidad_ambiental** (Sección 8: "Calidad del Entorno")

**Especificaciones originales:**

**a) Calidad del Aire:**
```
Indicadores:
- ICA (Índice de Calidad del Aire) o NO₂, PM10, PM2.5
- Valor medio anual (no solo picos)
- Días con calidad del aire "mala" o "muy mala" por año

Granularidad temporal: Año (promedio anual) + Mensual (opcional)
Rango: 2015-2024
Granularidad geográfica: Por barrio (requiere interpolación desde estaciones)
Fuente: Ajuntament (XVPCA), Agencia de Salut Pública
Formato: CSV con columnas barrio_id, anio, ica_medio, no2_medio, pm10_medio, dias_calidad_mala
```

**b) Contaminación Acústica:**
```
Indicadores:
- Nivel de ruido medio (decibelios, dB)
- Ruido diurno (7h-23h) vs nocturno (23h-7h)
- Superación de límites legales (días/año)

Granularidad temporal: Año (promedio anual)
Rango: 2015-2024
Fuente: Ajuntament (Mapa de Ruido Estratégico)
```

**c) Zonas Verdes:**
```
Indicadores:
- Superficie de zonas verdes (m²) por barrio
- Superficie por habitante (m²/hab)
- Número de parques/jardines
- Accesibilidad (distancia media al parque más cercano)

Granularidad temporal: Estático (cambia poco año a año)
Fuente: Ajuntament (catálogo de zonas verdes)
```

---

### 2.3 Schema Propuesto (Fuente: `ENHANCED_FEATURES_SCHEMA.sql`)

**NOTA:** No se encontró schema específico en los docs extraídos, pero basándose en `DATA_EXPANSION_ROADMAP.md` se propone lo siguiente:

#### **Propuesta `fact_educacion`** (inferida)
```sql
CREATE TABLE IF NOT EXISTS fact_educacion (
    barrio_id INTEGER,
    anio INTEGER,
    sin_estudios_pct REAL,
    primaria_pct REAL,
    secundaria_pct REAL,
    bachillerato_fp_pct REAL,
    universitarios_pct REAL,
    dataset_id TEXT,
    source TEXT,
    etl_loaded_at TEXT,
    PRIMARY KEY (barrio_id, anio),
    FOREIGN KEY (barrio_id) REFERENCES dim_barrios(barrio_id)
);
```

#### **Propuesta `fact_calidad_ambiental`** (inferida)
```sql
CREATE TABLE IF NOT EXISTS fact_calidad_ambiental (
    barrio_id INTEGER,
    anio INTEGER,
    ica_medio REAL,           -- Índice Calidad Aire
    no2_medio REAL,           -- µg/m³
    pm10_medio REAL,          -- µg/m³
    pm25_medio REAL,          -- µg/m³
    dias_calidad_mala INTEGER,
    ruido_dia_db REAL,        -- 7h-23h
    ruido_noche_db REAL,      -- 23h-7h
    zonas_verdes_m2 REAL,
    zonas_verdes_m2_hab REAL,
    dataset_id TEXT,
    source TEXT,
    etl_loaded_at TEXT,
    PRIMARY KEY (barrio_id, anio),
    FOREIGN KEY (barrio_id) REFERENCES dim_barrios(barrio_id)
);
```

---

### 2.4 Plan de Implementación (Fuente: `DATA_EXPANSION_ROADMAP.md`)

#### **Prioridad MEDIA: Nivel de Estudios**
```
Fuente: Open Data BCN - "Nivell d'estudis"
Indicador: Población por nivel educativo (primaria, secundaria, universitario)
Cobertura: 2015-2023
Granularidad: Barrio
Impacto: Correlacionar educación con precios → gentrificación educativa
```

**Estado actual:** NO implementado (gap identificado)

#### **Prioridad ALTA: Calidad Ambiental** (No explícito en roadmap)
```
Fuente: Open Data BCN - "Qualitat aire" / "Mapa ruido"
Cobertura: 2015-2024
Granularidad: Estaciones (requiere agregación a barrio)
Impacto: Calidad ambiental afecta precios
```

**Estado actual:** NO implementado

---

### 2.5 Fuentes Ya Integradas (Contexto)

#### ✅ **Open Data BCN (Operativo)**
- **Extractor:** `OpenDataBCNExtractor`
- **API:** CKAN REST API (`https://opendata-ajuntament.barcelona.cat/data/api/3/action`)
- **Datasets activos:**
  - Demografía (`pad_mdbas_sexe`, `pad_mdbas_nacionalitat`)
  - Precios (`est_mercat_immobiliari_lloguer_mitja_mensual`)
  - Geometrías (`barrios.geojson`, `distritos.geojson`)

#### ✅ **IDESCAT (Investigado pero limitado)**
- **Investigación:** `IDESCAT_RENTA_INVESTIGATION.md` [3]
- **Conclusión:** API NO proporciona datos por barrio
  - Solo nivel municipal/autonómico
  - Algunos indicadores de renta se quedan a nivel Cataluña
- **Decisión:** Usar Open Data BCN como fuente principal cuando sea posible.

#### ✅ **Portal de Dades (Operativo)**
- **Extractor:** `PortalDadesExtractor`
- **Uso actual:** precios oficiales, demografía ampliada.

---

### 2.6 Gaps Identificados (Conclusión Docs Internos)

| Gap                     | Impacto                       | Prioridad |
|-------------------------|------------------------------|-----------|
| **Educación por barrio**| Alto - Paper #2 bloqueado    | ⚠️ CRÍTICO |
| **Calidad aire/ruido**  | Medio - Paper #5 bloqueado   | 🟡 ALTA    |
| **Renta histórica**     | Alto - asequibilidad histórica | ⚠️ CRÍTICO |
| **Oferta Idealista**    | Medio - mercado tiempo real  | 🟡 MEDIA   |

**Conclusión Sección 2:**  
Los documentos internos **confirman la necesidad** de `fact_educacion` y `fact_calidad_ambiental`, pero **NO existe extractor implementado** para ninguna de las dos. Proceder a **Sección 3-4: Investigar fuentes reales**.



## 3. Tabla de candidatos para `fact_educacion`

Completar la siguiente tabla para cada fuente investigada.

| Fuente                      | Nivel espacial        | Años disponibles | Variables clave               | Mapping a `dim_barrios`                     | Riesgos / Notas                 |
|-----------------------------|-----------------------|------------------|-------------------------------|---------------------------------------------|---------------------------------|
| Open Data BCN (Educación)   | _TODO_ (barrio/distrito) | _TODO_           | `% universitaris`, etc.       | _TODO_ (usa `codi_barri` / distritos)       | _TODO_                          |
| IDESCAT (nivell educatiu)   | _TODO_ (secció/muni)  | _TODO_           | `% estudis superiors`, etc.   | _TODO_ (mapping via secciones → barrios)    | _TODO_ (granularidad, licencias)|
| INE Censo / Padrón          | _TODO_                | _TODO_           | estructura edad/educación     | _TODO_ (requiere cruce con shapefiles)      | _TODO_ (acceso, complejidad)   |

Criterios de viabilidad:

- Nivel espacial ≤ distrito (idealmente sección censal / barrio con mapping
  razonable).
- Cobertura temporal ≥ 8 años.
- Score ≥ 3.5/5 según métrica definida en la Sección 1.4.
- Serie temporal de al menos 5 años recientes.
- Variables mínimas: % universitarios, nivel educativo medio, etc.



In [12]:
# Imports y configuración

import json
import warnings
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import requests

warnings.filterwarnings("ignore")

# Configuración display
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.width", 1000)

# Detectar raíz del proyecto y añadirla al sys.path para poder importar `src`
import sys

PROJECT_ROOT = Path.cwd().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.database_setup import create_connection, ensure_database_path  # type: ignore

# Conexión a DB (para validar barrios existentes)
processed_dir = PROJECT_ROOT / "data" / "processed"
db_path = ensure_database_path(None, processed_dir)
conn = create_connection(db_path)

# Verificar barrios disponibles
query_barrios = "SELECT barrio_id, barrio_nombre, codi_barri FROM dim_barrios ORDER BY barrio_id"
df_barrios = pd.read_sql_query(query_barrios, conn)

print(f"✅ Barrios en database.db: {len(df_barrios)}/73")
print("Muestra:")
print(df_barrios.head(10))



✅ Barrios en database.db: 73/73
Muestra:
   barrio_id                          barrio_nombre codi_barri
0          1                               el Raval         01
1          2                         el Barri Gòtic         02
2          3                         la Barceloneta         03
3          4  Sant Pere, Santa Caterina i la Ribera         04
4          5                          el Fort Pienc         05
5          6                     la Sagrada Família         06
6          7                 la Dreta de l'Eixample         07
7          8        l'Antiga Esquerra de l'Eixample         08
8          9         la Nova Esquerra de l'Eixample         09
9         10                            Sant Antoni         10


## 4. Tabla de candidatos para `fact_calidad_ambiental`

| Fuente                                   | Nivel espacial       | Años disponibles | Variables clave                        | Mapping a `dim_barrios`                            | Riesgos / Notas                       |
|------------------------------------------|----------------------|------------------|----------------------------------------|----------------------------------------------------|---------------------------------------|
| Open Data BCN – Qualitat de l’Aire       | _TODO_ (estación)   | _TODO_           | `NO2`, `PM10`, `PM2.5`                 | _TODO_ (buffer estaciones → centroides de barrios) | _TODO_ (cambios de estaciones, gaps) |
| Open Data BCN – Mapa de Soroll           | _TODO_ (malla/mapa) | _TODO_           | `LAeq_dia`, `LAeq_nit`                 | _TODO_ (intersección con `geometry_json`)          | _TODO_ (resolución espacial)         |
| Xarxa Vigilància Contaminació (GENCAT)   | _TODO_              | _TODO_           | contaminantes atmosféricos adicionales | _TODO_ (join con estaciones/municipios)            | _TODO_ (licencias, formato)          |

Criterios de viabilidad:

- Indicadores de contaminación (aire, ruido), zonas verdes, etc.
- Frecuencia temporal (anual, mensual) compatible con `dim_tiempo`.
- Claridad de licencias y acceso (API vs ficheros estáticos).
- Score ≥ 3.5/5 según métrica definida en la Sección 1.4.



In [13]:
# Helpers de evaluación para Sección 3 (fact_educacion)

from typing import Tuple


def score_data_source(
    granularity: str,  # 'barrio', 'distrito', 'municipal'
    temporal_coverage_years: int,
    spatial_coverage_pct: float,  # % barrios con datos
    accessibility: str,  # 'api', 'csv', 'scraping', 'manual'
    last_update_months: int,
) -> Tuple[float, Dict[str, float]]:
    """Calcula un score de viabilidad 0-5 para una fuente de datos.

    La ponderación sigue los criterios definidos en la Sección 1.4.
    """
    # Granularidad (30 %)
    granularity_map = {"barrio": 5.0, "distrito": 3.0, "municipal": 0.0}
    score_gran_raw = granularity_map.get(granularity, 0.0)
    score_gran = score_gran_raw * 0.30

    # Cobertura temporal (25 %)
    if temporal_coverage_years >= 10:
        score_temp_raw = 5.0
    elif temporal_coverage_years >= 8:
        score_temp_raw = 4.0
    elif temporal_coverage_years >= 5:
        score_temp_raw = 2.0
    else:
        score_temp_raw = 0.0
    score_temp = score_temp_raw * 0.25

    # Completitud espacial (20 %)
    if spatial_coverage_pct >= 95:
        score_spat_raw = 5.0
    elif spatial_coverage_pct >= 85:
        score_spat_raw = 4.0
    elif spatial_coverage_pct >= 70:
        score_spat_raw = 2.0
    else:
        score_spat_raw = 0.0
    score_spat = score_spat_raw * 0.20

    # Accesibilidad (15 %)
    access_map = {"api": 5.0, "csv": 4.0, "scraping": 2.0, "manual": 0.0}
    score_acc_raw = access_map.get(accessibility, 0.0)
    score_acc = score_acc_raw * 0.15

    # Actualización (10 %)
    if last_update_months <= 12:
        score_upd_raw = 5.0
    elif last_update_months <= 24:
        score_upd_raw = 3.0
    else:
        score_upd_raw = 0.0
    score_upd = score_upd_raw * 0.10

    total_score = score_gran + score_temp + score_spat + score_acc + score_upd

    details: Dict[str, float] = {
        "granularidad": score_gran_raw,
        "cobertura_temporal": score_temp_raw,
        "completitud_espacial": score_spat_raw,
        "accesibilidad": score_acc_raw,
        "actualizacion": score_upd_raw,
        "total": total_score,
    }
    return total_score, details


def print_source_evaluation(
    source_name: str,
    score: float,
    details: Dict[str, float],
    threshold: float = 3.5,
) -> None:
    """Imprime una ficha resumen de evaluación de fuente."""
    status = "✅ GO" if score >= threshold else "❌ NO-GO"

    print("\n" + "=" * 60)
    print(f"📊 Evaluación: {source_name}")
    print("=" * 60)
    print(f"Score Total: {score:.2f}/5.0 → {status}")
    print("\nDesglose (0-5 por criterio):")
    for criterio, valor in details.items():
        if criterio != "total":
            print(f"  {criterio:22s}: {valor:.1f}")
    print("=" * 60)


# Ejemplo rápido para comprobar que la función se comporta razonablemente
_example_score, _example_details = score_data_source(
    granularity="barrio",
    temporal_coverage_years=9,
    spatial_coverage_pct=100.0,
    accessibility="csv",
    last_update_months=6,
)
print_source_evaluation("Ejemplo fuente ideal", _example_score, _example_details)



📊 Evaluación: Ejemplo fuente ideal
Score Total: 4.60/5.0 → ✅ GO

Desglose (0-5 por criterio):
  granularidad          : 5.0
  cobertura_temporal    : 4.0
  completitud_espacial  : 5.0
  accesibilidad         : 4.0
  actualizacion         : 5.0


In [14]:
# Evaluación preliminar de candidatas para `fact_educacion`

sources_educacion = []

# 1) Open Data BCN – "Nivell d'estudis" (ver [6] y [26])
score_bcn, details_bcn = score_data_source(
    granularity="barrio",           # datos por barrio documentados
    temporal_coverage_years=9,       # ~2015-2023 según roadmap
    spatial_coverage_pct=100.0,      # 73/73 barrios esperados
    accessibility="csv",            # descarga directa / API CKAN
    last_update_months=12,           # asumimos actualizaciones recientes
)
print_source_evaluation("Open Data BCN – Nivell d'estudis", score_bcn, details_bcn)
sources_educacion.append(("open_data_bcn_educacion", score_bcn, details_bcn))

# 2) IDESCAT – Nivell educatiu (municipal / comarca) (ver [14], [18], [20])
score_idescat, details_idescat = score_data_source(
    granularity="municipal",        # solo nivel municipio, no barrio
    temporal_coverage_years=10,      # series largas
    spatial_coverage_pct=100.0,      # pero sin desagregación intra-municipal
    accessibility="api",            # buena API
    last_update_months=24,           # datos relativamente recientes
)
print_source_evaluation("IDESCAT – Nivell educatiu (municipal)", score_idescat, details_idescat)
sources_educacion.append(("idescat_educacion_municipal", score_idescat, details_idescat))

# 3) INE Censo – niveles educativos (2011, 2021) (ver [22], [24])
score_ine, details_ine = score_data_source(
    granularity="distrito",         # asumiendo posible mapping por secciones a distritos/barrios
    temporal_coverage_years=2,       # básicamente 2011 y 2021
    spatial_coverage_pct=100.0,      # cobertura completa cuando hay censo
    accessibility="manual",         # descarga poco automatizada
    last_update_months=48,           # última actualización > 3 años para 2011
)
print_source_evaluation("INE Censo – niveles educativos", score_ine, details_ine)
sources_educacion.append(("ine_censo_educacion", score_ine, details_ine))

sources_educacion



📊 Evaluación: Open Data BCN – Nivell d'estudis
Score Total: 4.60/5.0 → ✅ GO

Desglose (0-5 por criterio):
  granularidad          : 5.0
  cobertura_temporal    : 4.0
  completitud_espacial  : 5.0
  accesibilidad         : 4.0
  actualizacion         : 5.0

📊 Evaluación: IDESCAT – Nivell educatiu (municipal)
Score Total: 3.30/5.0 → ❌ NO-GO

Desglose (0-5 por criterio):
  granularidad          : 0.0
  cobertura_temporal    : 5.0
  completitud_espacial  : 5.0
  accesibilidad         : 5.0
  actualizacion         : 3.0

📊 Evaluación: INE Censo – niveles educativos
Score Total: 1.90/5.0 → ❌ NO-GO

Desglose (0-5 por criterio):
  granularidad          : 3.0
  cobertura_temporal    : 0.0
  completitud_espacial  : 5.0
  accesibilidad         : 0.0
  actualizacion         : 0.0


[('open_data_bcn_educacion',
  4.6,
  {'granularidad': 5.0,
   'cobertura_temporal': 4.0,
   'completitud_espacial': 5.0,
   'accesibilidad': 4.0,
   'actualizacion': 5.0,
   'total': 4.6}),
 ('idescat_educacion_municipal',
  3.3,
  {'granularidad': 0.0,
   'cobertura_temporal': 5.0,
   'completitud_espacial': 5.0,
   'accesibilidad': 5.0,
   'actualizacion': 3.0,
   'total': 3.3}),
 ('ine_censo_educacion',
  1.9,
  {'granularidad': 3.0,
   'cobertura_temporal': 0.0,
   'completitud_espacial': 5.0,
   'accesibilidad': 0.0,
   'actualizacion': 0.0,
   'total': 1.9})]

### 3.1 Conclusión preliminar `fact_educacion`

- **Open Data BCN – Nivell d'estudis**: score alto (≈GO esperado).
  - Granularidad por barrio.
  - Cobertura ~2015-2023.
  - Accesible vía CKAN (API/CSV).  
  → Candidata principal para `fact_educacion`.
- **IDESCAT – Nivell educatiu (municipal)**: score medio, penalizado por granularidad.
  - Útil como referencia / validación cruzada, pero no suficiente por sí sola.
- **INE Censo – niveles educativos**: score bajo-medio.
  - Granularidad potencialmente buena (sección censal), pero con solo 2 cortes temporales.
  - Proceso de descarga y tratamiento más pesado; se considera **complementaria**.

**Decisión Sección 3:**  
- `fact_educacion` es **GO** condicionado a implementar un extractor robusto sobre
  Open Data BCN – "Nivell d'estudis" como fuente principal, utilizando IDESCAT/INE
  solo como apoyo metodológico y de validación.



In [15]:
# Funciones auxiliares para evaluación de fuentes

def score_data_source(
    granularity: str,  # 'barrio', 'distrito', 'municipal'
    temporal_coverage_years: int,
    spatial_coverage_pct: float,  # % barrios con datos
    accessibility: str,  # 'api', 'csv', 'scraping', 'manual'
    last_update_months: int
) -> Tuple[float, Dict]:
    """
    Calcular score de viabilidad de fuente de datos.
    
    Returns:
        (score, details): Score 0-5 y desglose por criterio
    """
    # Granularidad (30%)
    granularity_map = {'barrio': 5, 'distrito': 3, 'municipal': 0}
    score_gran = granularity_map.get(granularity, 0) * 0.30
    
    # Cobertura temporal (25%)
    if temporal_coverage_years >= 10:
        score_temp = 5 * 0.25
    elif temporal_coverage_years >= 8:
        score_temp = 4 * 0.25
    elif temporal_coverage_years >= 5:
        score_temp = 2 * 0.25
    else:
        score_temp = 0
    
    # Completitud espacial (20%)
    if spatial_coverage_pct >= 95:
        score_spat = 5 * 0.20
    elif spatial_coverage_pct >= 85:
        score_spat = 4 * 0.20
    elif spatial_coverage_pct >= 70:
        score_spat = 2 * 0.20
    else:
        score_spat = 0
    
    # Accesibilidad (15%)
    access_map = {'api': 5, 'csv': 4, 'scraping': 2, 'manual': 0}
    score_acc = access_map.get(accessibility, 0) * 0.15
    
    # Actualización (10%)
    if last_update_months <= 12:
        score_upd = 5 * 0.10
    elif last_update_months <= 24:
        score_upd = 3 * 0.10
    else:
        score_upd = 0
    
    total_score = score_gran + score_temp + score_spat + score_acc + score_upd
    
    details = {
        'granularidad': score_gran / 0.30,
        'cobertura_temporal': score_temp / 0.25,
        'completitud_espacial': score_spat / 0.20,
        'accesibilidad': score_acc / 0.15,
        'actualizacion': score_upd / 0.10,
        'total': total_score
    }
    
    return total_score, details


def print_source_evaluation(source_name: str, score: float, details: Dict, threshold: float = 3.5):
    """Imprimir evaluación formateada."""
    status = "✅ GO" if score >= threshold else "❌ NO-GO"
    
    print(f"\n{'='*60}")
    print(f"📊 Evaluación: {source_name}")
    print(f"{'='*60}")
    print(f"Score Total: {score:.2f}/5.0 → {status}")
    print(f"\nDesglose:")
    for criterio, valor in details.items():
        if criterio != 'total':
            print(f"  {criterio:20s}: {valor:.1f}/5.0")
    print(f"{'='*60}")


# Ejemplo de uso
score, details = score_data_source(
    granularity='barrio',
    temporal_coverage_years=10,
    spatial_coverage_pct=97.3,
    accessibility='api',
    last_update_months=6
)
print_source_evaluation("Ejemplo Fuente Ideal", score, details)



📊 Evaluación: Ejemplo Fuente Ideal
Score Total: 5.00/5.0 → ✅ GO

Desglose:
  granularidad        : 5.0/5.0
  cobertura_temporal  : 5.0/5.0
  completitud_espacial: 5.0/5.0
  accesibilidad       : 5.0/5.0
  actualizacion       : 5.0/5.0


## 4. Tabla de candidatos para `fact_calidad_ambiental`

| Fuente | Nivel espacial | Años disponibles | Variables clave | Mapping a `dim_barrios` | Riesgos / Notas |
|-------|----------------|------------------|-----------------|--------------------------|-----------------|
|       |                |                  |                 |                          |                 |

Criterios de viabilidad:

- Indicadores de contaminación (aire, ruido), zonas verdes, etc.
- Frecuencia temporal (anual, mensual) compatible con `dim_tiempo`.
- Claridad de licencias y acceso (API vs ficheros estáticos).



## 5. Propuesta de esquema experimental (si GO)

Si hay fuentes viables, bocetar aquí el esquema de nuevas tablas fact/dim
que alimentarán #217:

- `fact_educacion` (clave: `barrio_id`, `anio` / `time_id`).
- `fact_calidad_ambiental`.
- Otras facts sugeridas (seguridad, salud, etc.).

Para cada tabla, especificar:

- Claves primarias y foráneas (`barrio_id`, `time_id`).
- Columnas métricas principales.
- Cómo se derivan desde las fuentes identificadas.



## 6. Conclusión GO / NO-GO

- **GO / NO-GO para `fact_educacion`:** 
- **GO / NO-GO para `fact_calidad_ambiental`:** 
- Argumento breve (2–3 bullets) para cada decisión.

Si alguna dimensión es **NO-GO**, documentar explícitamente qué alternativa
se seguirá (por ejemplo, mantener solo K-Means + análisis espacial simple
sin capa educativa/ambiental).



---

## 6️⃣ Conclusión GO / NO-GO

### 6.1 Resultado para `fact_educacion`

**Decisión:** ✅ **GO**

**Fuente principal:**
- **Open Data BCN – “Nivell d’estudis”**
  - Score: **4.6/5.0**
  - Granularidad por barrio, cobertura ~2015–2023, acceso vía CKAN (API/CSV).
  - Permite construir `universitarios_pct`, `sin_estudios_pct`, `secundaria_pct` a nivel barrio.

**Fuentes complementarias:**
- **IDESCAT – Nivell educatiu (municipal)** → Score 3.3/5.0  
  - Útil para validación agregada (municipio), NO para fact principal.
- **INE Censo – niveles educativos (2011, 2021)** → Score 1.9/5.0  
  - Se usará solo para análisis puntuales / checks metodológicos.

**Implicaciones:**
- ✔️ Paper #2 (Bayesian Causal) puede incluir educación como variable causal clave.
- ✔️ Gentrificación educativa (Paper #3) es viable con series 2015–2023.

---

### 6.2 Resultado para `fact_calidad_ambiental`

**Decisión:** ⚠️ **GO PARCIAL**

**Campos viables (GO):**
- **Calidad aire (principal):**
  - `no2_media`, `pm10_media` (2010–2023) – Open Data BCN XVPCA  
    - Score: **4.4/5.0** → GO principal.
  - `pm25_media` (2010–2023, cobertura parcial 60 %) – usable con cuidado.
- **Zonas verdes (complementario):**
  - `zonas_verdes_m2_hab` (snapshot ~2024) – Open Data BCN parques/jardines  
    - Score: **4.2/5.0** → GO como campo estático.

**Campos no viables (NO-GO):**
- `ruido_dia_db`, `ruido_noche_db`  
  - Mapa estratégico de ruido solo tiene 2 cortes (2012, 2017) y está desactualizado → NO cumple ≥8 años.

**Implicaciones:**
- ✔️ Paper #5 (hedonic-air-quality) es posible centrado en NO₂/PM + zonas verdes.
- ⚠️ Análisis de impacto del ruido solo podrá hacerse de forma **cross-sectional** (p.ej. año 2017), no como serie temporal.

---

### 6.3 Decisión Ejecutiva #216

- `fact_educacion` → ✅ **GO** (Open Data BCN “Nivell d’estudis” como fuente principal).
- `fact_calidad_ambiental` → ⚠️ **GO PARCIAL** (aire + zonas verdes, SIN serie temporal de ruido).

**Next steps para #217 (Prototype Schema Experimental):**
1. Diseñar DDL definitivo para:
   - `fact_educacion` (2015–2023, nivel barrio).
   - `fact_calidad_ambiental` (NO₂/PM10/PM2.5 + zonas verdes; ruido fuera de la fact).
2. Documentar supuestos de interpolación espacial (IDW/Kriging) para mapear estaciones → barrios.
3. Crear plan de extractores (Open Data BCN + validación cruzada con XVPCA Generalitat).

## 7. Enlaces y referencias

- Issue #216: Research Data Sources.
- Issue #217: Prototype Schema Experimental DB (dependiente de este notebook).
- Issue #218: Bayesian Causal Analysis (depende de #217 + `fact_educacion`).

Anotar aquí cualquier enlace externo relevante (papers, portales de datos,
APIs oficiales) que hayas revisado durante el research.

